### Test 2

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
import csv

truth = []
with open('train.truth.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        truth.append(row)

In [3]:
import glob
import cv2

#im_test = [cv2.imread(file) for file in glob.glob("test/*.jpg")]
im_test0 = []
pred = [['fn', 'label']]
for file in glob.glob("test/*.jpg"):
    pred.append([file[5:], ''])
    im_test0.append(cv2.imread(file))
    
im_test = np.zeros((len(im_test0), 64, 64, 3), dtype=np.uint8)
for i in range(len(im_test0)):
    im_test[i, :, :, :] = im_test0[i]

# im_train = []
# labels = []
im_train = np.zeros((len(truth), 64, 64, 3), dtype=np.uint8)
labels = np.zeros((len(truth), 4), dtype=np.float32)
label_dict = {'upright': 0, 'rotated_left': 1, 'rotated_right': 2, 'upside_down':3}
for i, elem in enumerate(truth[1:]):
    file_name = 'train/' + elem[0]
    im_train[i, :, :, :] = cv2.imread(file_name)
    j = label_dict[elem[1]]
    labels[i, j] = 1 
#     im_train.append(cv2.imread(file_name))
#     labels.append(label_dict[elem[1]])

In [4]:
im_train = im_train.astype('float32')
im_test = im_test.astype('float32')
im_train /= 255
im_test /= 255

In [5]:
im_train.shape

(48897, 64, 64, 3)

In [6]:
labels.shape

(48897, 4)

In [7]:
# The data, split between train and test sets:
from sklearn.model_selection import train_test_split

im_train = im_train[0:10000, :, :, :]
labels = labels[0:10000, :]
x_train, x_test, y_train, y_test = train_test_split(im_train, labels, test_size=0.2)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (8000, 64, 64, 3)
8000 train samples
2000 test samples


In [8]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 64
num_classes = 4#10
epochs = 2#100


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# Save model and weights
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/2
125/125 [==============================] - 76s 607ms/step - loss: 1.1850 - accuracy: 0.4823 - val_loss: 0.8168 - val_accuracy: 0.7360
Epoch 2/2
63/63 [==============================] - 3s 52ms/step - loss: 0.5324 - accuracy: 0.8125
Test loss: 0.5323801636695862
Test accuracy: 0.8125


### Predict on test images and save csv file

In [18]:
# Predict on Test images
im_test = im_test[0:1000, :, :, :]
model_predict = model.predict(im_test)

for i, elem in enumerate(pred[1:]):
    argmax = np.argmax(model_predict[i, :])
    if argmax == 0:
        elem[1] = 'upright'
    elif argmax == 1:
        elem[1] = 'rotated_left'
    elif argmax == 2:
        elem[1] = 'rotated_right'
    else:
        elem[1] = 'upside_down'
        
pred[0:5]

[['fn', 'label'],
 ['93-23730893_1988-05-18_2012.jpg', 'rotated_left'],
 ['99-667599_1938-06-05_1963.jpg', 'upright'],
 ['99-45240299_1991-05-22_2014.jpg', 'rotated_right'],
 ['92-292592_1967-02-19_2014.jpg', 'rotated_right']]

In [3]:
with open('test.preds.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for elem in pred:
        writer.writerow(elem)

NameError: name 'csv' is not defined